<a href="https://colab.research.google.com/github/ubsuny/PHY386/blob/Homework2025/2025/HW/william-dove/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the solution provided for the 3-mass, 4-spring case:

In [3]:
# 3 mass soln

# import libraries
import numpy as np


# Let's calculate the frequencies numerically:

def calculate_eigenfrequencies_3mass(k, m):
    """ calculate the eigenfrequencies for our three mass problem.

    Args:
    k -- spring constant (N/m)
    m -- mass (kg)

    Returns:
    w1 -- first eigenfrequency
    w2 -- second eigenfrequency
    w3 -- third eigenfrequency
    """
    w0_squared = k / m
    w1 = np.sqrt(2*w0_squared)
    w2 = np.sqrt((2 - np.sqrt(2)) * w0_squared)
    w3 = np.sqrt((2 + np.sqrt(2)) * w0_squared)
    return w1, w2, w3

# Example values
k = 10  # N/m
m = 1   # kg

w1, w2, w3 = calculate_eigenfrequencies_3mass(k, m)

print("Eigenfrequencies:")
print("ω₁ = {:.4f} rad/s".format(w1))
print("ω₂ = {:.4f} rad/s".format(w2))
print("ω₃ = {:.4f} rad/s".format(w3))

Eigenfrequencies:
ω₁ = 4.4721 rad/s
ω₂ = 2.4203 rad/s
ω₃ = 5.8431 rad/s


This is the solution for *n* masses, and either *n* or *n*+1 springs, depending on whether the system is attached to walls on both ends or just one:

First, we define functions to create the mass and stiffness matrices.

In [4]:
def setup_mass_matrix(m_list):
  '''
  This function creates a matrix of masses using the provided masses list as the diagnoal terms.

  Args:
  m_list: list of masses (kg)

  Returns:
  mass_matrix: mass matrix (numpy array)
  '''
  M = np.diag(m_list)
  return M

print(setup_mass_matrix([1,1,1]))

[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [5]:
def setup_stiffness_matrix(k_list):
  '''
  This function creates a matrix of spring constants using the provided spring constants list. The diagonal terms are defined as -(k_n+k_n+1),
  and the off-diagonal terms are defined as k_n+1.

  Args:
  k_list: list of spring constants (N/m)

  Returns:
  stiffness_matrix: stiffness matrix (numpy array)
  '''
  stiffness_matrix = np.diag([-k_list[i] - k_list[i+1] for i in range(len(k_list)-1)])
  stiffness_matrix_offDiags = np.diag(k_list[1:-1], k=+1) + np.diag(k_list[1:-1], k=-1)
  stiffness_matrix += stiffness_matrix_offDiags
  return stiffness_matrix

print(setup_stiffness_matrix([10,10,10,10]))

[[-20  10   0]
 [ 10 -20  10]
 [  0  10 -20]]


Next, we can use these matrices and the SciPy library for linear algebra to find the eigenfrequencies.

We assume that $x=e^{i\omega x}$ so that $\frac{d^2 x}{dx^2}=-\omega^2 x$. Then we can use $M(-\omega^2)x=Kx \rightarrow M^{-1}Kx=-\omega^2 x$. The eigenvalues of the $M^{-1}K$ matrix are $-\omega^2$ so we can solve for the frequencies.

In [12]:
# Code extended for n masses with unique mass and n+2 springs with unique spring constants.

import scipy.linalg as la

def calculate_eigenfrequencies_nmass(k_list, m_list):
    """
    Calculates the eigenfrequencies for a system with n masses and either n or n+1 springs.
    Each mass and spring constant may be unique. Input masses and spring constants as lists.

    Args:
    k_list: list of spring constants (N/m)
    m_list: list of masses (kg)

    Returns:
    omega_list: list of eigenfrequencies (rad/s)
    """
    M = setup_mass_matrix(m_list)
    K = setup_stiffness_matrix(k_list)
    MinvK = la.inv(M)*K
    eigs_list = la.eigh(MinvK)[0]
    omega_list = np.sqrt(-eigs_list)
    return omega_list

print(calculate_eigenfrequencies_nmass([10,10,10,10],[1,1,1]))

[4.47213595 4.47213595 4.47213595]


Comparing the results from the provided/extended code:

In [24]:
from IPython.display import display, Math

# Example values
k = 10  # N/m
m = 1   # kg
k_list = [10,10,10,10]
m_list = [1,1,1]

# Provided 3 mass code
w1, w2, w3 = calculate_eigenfrequencies_3mass(k, m)
print("Provided Result:")
print("ω₁ = {:.4f} rad/s".format(w1))
print("ω₂ = {:.4f} rad/s".format(w2))
print("ω₃ = {:.4f} rad/s".format(w3))

# My n mass code
my_result = calculate_eigenfrequencies_nmass(k_list, m_list)
print('\nMy Result:')

display(*[Math(rf'\omega_{{{n+1}}} = {my_result[n]:.4f} \text{{ rad/s}}') for n in range(len(my_result))])

Provided Result:
ω₁ = 4.4721 rad/s
ω₂ = 2.4203 rad/s
ω₃ = 5.8431 rad/s

My Result:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>